In [ ]:
#workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
import os
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")

print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import esmfRegrid as erg

importlib.reload(erg)
importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
#############################################
# Conservative remapping of Topo PHIS
############################################################
############################################################

Src,Dst = 'ne240pg3','latlonOxO'

DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
dstHkey = DstInfo['Hkey']
dst_type =DstInfo['type']
dst_scrip =DstInfo['scrip']
#dst_TopoFile = DstInfo['TopoFile']

lat_x,lon_x = GrU.latlon(dst_scrip,dstHkey)
ny,nx=len(lat_x),len(lon_x)

SrcInfo = GrU.gridInfo(Src)
srcHkey = SrcInfo['Hkey']
src_type =SrcInfo['type']
src_scrip =SrcInfo['scrip']
src_TopoFile = SrcInfo['TopoFile']
print( f"Used NEW, concise gridInfo function .... ...." )
print( f"{src_TopoFile}")
print( f"{src_scrip} ==> {dst_scrip}")
print( f"{ny,nx}")


In [ ]:
Topo_src = xr.open_dataset( src_TopoFile )
topo_src = Topo_src.PHIS.values / 9.8
plt.plot( topo_src )

In [ ]:
%%time

RegridMethod = "CONSERVE_2ND"
regrd, srcF, dstF = erg.Regrid( srcScrip = src_scrip , 
                                srcType  = src_type  ,
                                dstScrip = dst_scrip ,
                                dstType  = dst_type  ,
                                RegridMethod = RegridMethod , read_weights=True, weights_file= "../AnimationFiles/ne240_OxO_conv2nd_remap.nc")

In [ ]:
#topo_dst = np.zeros( (nt,ny,nx) , dtype=np.float64 )
topo_dst = erg.HorzRG( aSrc = topo_src , 
                    regrd = regrd , 
                    srcField= srcF , 
                    dstField= dstF , 
                    srcGridkey= srcHkey ,
                    dstGridkey= dstHkey )

In [ ]:
dims   = ["lon","lat"]
coords = dict( 
    lon  = ( ["lon"],lon_x ),
    lat  = ( ["lat"],lat_x ),
)

Topo_dst = xr.Dataset( coords=coords  )

Dar = xr.DataArray( data=topo_dst, dims=('lat','lon',),
                    attrs=dict( description='height',units='meters',) ,) 
Topo_dst["TopoHt"]=Dar
Topo_dst.to_netcdf("../AnimationFiles/Topo_ne240_OxO_remap.nc" )